# 爬取大盤


自從上一堂課開始後，同學開始想要新增新的數據到資料庫，但是其實有點難，

這個單元將會分享如何新增爬蟲，並且將爬蟲加入 Finlab 所提供的 API 中

### 爬蟲的網址

我們想要爬取的網址如下：
http://www.tse.com.tw/zh/page/trading/exchange/MI_5MINS_INDEX.html

### 下載某天的資料

In [ ]:
import requests

import datetime

date = datetime.date(2020,3,19)
datestr = date.strftime("%Y%m%d")

query_string = {
 'response': 'csv',
 'date': datestr,
 '&_': 1584615922052
}

res = requests.get("https://www.twse.com.tw/exchangeReport/MI_5MINS_INDEX", params=query_string)

### 將資料用pandas整理成表格(dataframe)

Pandas 教學：
http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

In [29]:
import pandas as pd
import io
df = pd.read_csv(io.StringIO(res.text.replace("=", "")), header=1, index_col='時間')
# df.head()

,發行量加權股價指數,未含金融保險股指數,未含電子股指數,未含金融電子股指數,水泥類指數,食品類指數,塑膠類指數,紡織纖維類指數,電機機械類指數,電器電纜類指數,...,資訊服務類指數,其他電子類指數,建材營造類指數,航運類指數,觀光類指數,金融保險類指數,貿易百貨類指數,油電燃氣類指數,其他類指數,Unnamed: 35
時間,,,,,,,,,,,,,,,,,,,,,
09:00:00,"9,218.67","7,761.77","11,584.55","9,354.19",138.93,"1,487.63",185.13,386.66,141.91,38.66,...,100.70,67.66,238.41,47.74,64.91,"1,089.87",226.37,89.64,199.21,NaN
09:00:05,"9,085.28","7,658.16","11,449.12","9,289.69",136.66,"1,487.59",180.21,386.60,141.68,38.64,...,100.70,66.81,238.24,47.36,64.65,"1,066.69",226.34,89.64,198.93,NaN
09:00:10,"9,030.24","7,613.89","11,352.34","9,207.95",134.13,"1,481.70",177.46,381.74,140.88,38.34,...,100.47,66.60,238.24,46.91,64.50,"1,058.42",226.32,88.18,196.41,NaN
09:00:15,"9,014.92","7,600.31","11,319.19","9,173.04",134.03,"1,485.72",177.33,381.88,140.61,38.32,...,100.47,66.52,235.81,46.91,64.50,"1,057.19",223.20,88.18,194.55,NaN
09:00:20,"9,002.90","7,589.19","11,302.84","9,155.71",134.13,"1,482.98",177.18,381.51,140.07,38.39,...,98.76,66.42,235.69,46.65,64.33,"1,056.62",223.50,88.26,193.85,NaN


### 資料處理
1. 去除不需要的行、列
2. 設定index為日期
3. 將逗點「,」刪除，讓table裡面的字串(str)變成數字(float or int)

In [31]:
df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)

df.index = pd.to_datetime(datestr + ' ' + df.index)

In [ ]:
# one line lambda
# df = df.applymap(lambda s: float(str(s).replace(",", "")))

# 具名函數
#float(str(e).replace(",", ""))

# def abc(e):
#     ret = str(e)
#     ret = ret.replace(",", "")
#     ret = float(ret)
#     return ret

def abc(value):
    value_str = str(value)
    no_comma = value_str.replace(",", "")
    return float(no_comma)
df = df.applymap(abc)
df['2020-03-19 09:00': '2020-03-19 10:00:00']